In [20]:
import numpy as np
import random 
import sys

In [2]:
with open('poems.txt', 'r', encoding='utf-8') as file: 
    text = file.read() 

print(text) 

﻿Buffalo Bill’s
defunct
who used to
ride a watersmooth-silver
stallion
and break one two three four five pigeons just like that
Jesus

he was a handsome man
and what i want to know is
how do you like your blueeyed boy
Mister Death

Had I the heaven’s embroidered cloths,
Enwrought with golden and silver light,
The blue and the dim and the dark cloths
Of night and light and the half-light,
I would spread the cloths under your feet:
But I, being poor, have only my dreams;
I have spread my dreams under your feet;
Tread softly because you tread on my dreams.

He clasps the crag with crooked hands;
Close to the sun in lonely lands,
Ring’d with the azure world, he stands.

The wrinkled sea beneath him crawls;
He watches from his mountain walls,
And like a thunderbolt he falls.

Some say the world will end in fire,
Some say in ice.
From what I’ve tasted of desire
I hold with those who favor fire.
But if it had to perish twice,
I think I know enough of hate
To say that for destruction ice
Is al

In [3]:
unique_char=sorted(list(set(text))) 
  
char_to_indices = dict((c, i) for i, c in enumerate(unique_char)) 
indices_to_char = dict((i, c) for i, c in enumerate(unique_char)) 
  
print(unique_char) 

['\n', ' ', '!', ',', '-', '.', ':', ';', 'A', 'B', 'C', 'D', 'E', 'F', 'H', 'I', 'J', 'M', 'O', 'R', 'S', 'T', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z', '—', '’', '\ufeff']


In [6]:
max_length = 100
steps = 5
sentences = [] 
next_chars = [] 
for i in range(0, len(text) - max_length, steps): 
    sentences.append(text[i: i + max_length]) 
    next_chars.append(text[i + max_length]) 

X = np.zeros((len(sentences), max_length, len(unique_char)), dtype = np.bool_) 
y = np.zeros((len(sentences), len(unique_char)), dtype = np.bool_) 
  

for i, sentence in enumerate(sentences): 
    for t, char in enumerate(sentence): 
        X[i, t, char_to_indices[char]] = 1
    y[i, char_to_indices[next_chars[i]]] = 1

In [11]:
import tensorflow as tf 
from keras.models import Sequential 
from keras.layers import Dense, Activation 
from keras.layers import GRU
from keras.optimizers import RMSprop 

In [13]:
model = Sequential() 
model.add(GRU(128, input_shape =(max_length, len(unique_char)))) 
model.add(Dense(len(unique_char))) 
model.add(Activation('softmax'))  
optimizer = RMSprop(learning_rate= 0.01) 
model.compile(loss ='categorical_crossentropy', optimizer = optimizer) 

In [14]:
def sample_index(preds, temperature = 1.0): 
    
    preds = np.asarray(preds).astype('float64') 
    preds = np.log(preds) / temperature 
    exp_preds = np.exp(preds) 
    preds = exp_preds / np.sum(exp_preds)  
    probas = np.random.multinomial(1, preds, 1) 

    return np.argmax(probas) 

In [22]:
from keras.callbacks import LambdaCallback 

def on_epoch_end(epoch, logs): 
    print() 
    print('----- Generating text after Epoch: % d' % epoch) 
    start_index = random.randint(0, len(text) - max_length - 1) 
    
    for diversity in [0.2, 0.5, 1.0, 1.2]: 
        print('----- diversity:', diversity) 
  
        generated = '' 
        sentence = text[start_index: start_index + max_length] 
  
        generated += sentence 
        print('----- Generating with seed: "' + sentence + '"') 
        sys.stdout.write(generated) 
  
        for i in range(400): 
        
            x_pred = np.zeros((1, max_length, len(unique_char))) 
            for t, char in enumerate(sentence): 
                x_pred[0, t, char_to_indices[char]] = 1. 
            preds = model.predict(x_pred, verbose = 0)[0] 
            next_index = sample_index(preds, diversity) 
            next_char = indices_to_char[next_index] 
            generated += next_char 
            sentence = sentence[1:] + next_char 
  
            sys.stdout.write(next_char) 
            sys.stdout.flush() 
        print() 
        
print_callback = LambdaCallback(on_epoch_end = on_epoch_end) 

In [23]:
from keras.callbacks import ModelCheckpoint 

filepath = "weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor ='loss', verbose = 1, save_best_only = True, mode ='min') 

In [24]:
from keras.callbacks import ReduceLROnPlateau 

reduce_alpha = ReduceLROnPlateau(monitor ='loss', factor = 0.2, patience = 1, min_lr = 0.001) 
callbacks = [print_callback, checkpoint, reduce_alpha] 

In [26]:
model.fit(X, y, batch_size = 128, epochs = 15, callbacks = callbacks) 

Epoch 1/15
2/3 [===================>..........] - ETA: 0s - loss: 2.8012
----- Generating text after Epoch:  0
----- diversity: 0.2
----- Generating with seed: "our five pigeons just like that
Jesus

he was a handsome man
and what i want to know is
how do you l"
our five pigeons just like that
Jesus

he was a handsome man
and what i want to know is
how do you l t t e  t e  e  t e  e  e  t  t ee nd t e  t t e  t  t t t  e  t  t e e te te   t  te    t  t e t t t  t  e  t e  e  t  t   t t e  t    t  t t t  t t  t t e t we  t  t  t  t t te  t e e  t e te t  e e   t  t ee    t  t ee t  e  t e w   t te e t  t t e  t w t  t  t te t  t a  t  e  t ee  e   t e  t e te t t t   e te t  t   t e t t te te ee  te e  t  e  t t    t e  t te t  a  t t  t t  t e  t t e e 
----- diversity: 0.5
----- Generating with seed: "our five pigeons just like that
Jesus

he was a handsome man
and what i want to know is
how do you l"
our five pigeons just like that
Jesus

he was a handsome man
and what i want to know

mnngaffTwv loluv
Johl 
OsrsOnO t;alT g ;e l  c ed S handsMwwyou
a sc sOyr we.Ocysvut.
I aen g
B wsn o neou gue 
 ae wpDs
;fahBTinwon ntseB h C unus mer   se fnmhe  dkTI Buss
s
Srd raecsf
pe hecrrt 
.d wa 
cnd’mTpmyTn th uuvw thvwhc u I uyno hcem s wpded pn wAmvT TI cwr
je isc d
ja.ba leRlI s
O;m;
Ta l

Epoch 3: loss improved from 2.56108 to 2.43520, saving model to weights.hdf5
3/3 [==============================] - 61s 31s/step - loss: 2.4352 - lr: 0.0100
Epoch 4/15
2/3 [===================>..........] - ETA: 0s - loss: 2.3551
----- Generating text after Epoch:  3
----- diversity: 0.2
----- Generating with seed: "re
I hold with those who favor fire.
But if it had to perish twice,
I think I know enough of hate
To"
re
I hold with those who favor fire.
But if it had to perish twice,
I think I know enough of hate
To t te t t te t t t t t te t t te t t s t t t t t t t t te te t t t te t te t t t t t t t t t t t t t t t t t t t t t t t t t te t t t t t t t t t t t t t t t t t t t t t t t t 

Twsy we rsMeyhe i l ve eS;ve y sy cee gobeerjceads oOoyeve moe , Ioldg w fuy te wlyd ld Fl: lde c—i

deed g tseme fwkouyologerirkee
Imede nd
----- diversity: 1.2
----- Generating with seed: "ands.

The wrinkled sea beneath him crawls;
He watches from his mountain walls,
And like a thunderbo"
ands.

The wrinkled sea beneath him crawls;
He watches from his mountain walls,
And like a thunderbo beccfe woyoCe pd pend eas;e haRuryeM..v
.Tyoeece.
:
wlmdoyoy lekfoyJde ty wous﻿neeop t odcpe !;j
I’y bdea;cnbgher;erf;,Trf—wos’d IeaSysfM

Titepe
 i bqlhy M taemreren
kemyq
Suw ;r;I  ha﻿rr  eape wnde ip f gyesvvje Ie S!ver w I;anm feeey dieg chy
Ohefebr dakkg foos﻿ke
;-hi.
Awomee y
.i﻿e e I TpeAng
’f mee.r Tw
uve aSokey tez;zSfme y sfbf!IAndeyeherdiie bld DOuir ls;o’ toubd eanr TpeumeahyeoCeotr e

Epoch 6: loss improved from 2.11820 to 1.98666, saving model to weights.hdf5
3/3 [==============================] - 62s 31s/step - loss: 1.9867 - lr: 0.0100
Epoch 7/15
2/3 [===================>..........] 

I have spr ly ir, 
as tlde wheSi asCe an ls; 
lvkesund te wraf thuvire fre roes ge Is s’ wher ceyume on tf
And ron t he alsp ls y tas s lirs ps ce uravE fow ag fde te he w ay che Sa  wane tls;elclld  wonwkset hant ly he I thas the hirfads h nbousas fallsry.
I Twhire I cuis .
Ih Busg whus the I  bau ggeI ihe ck ce tesam thurkonde punbey
 ha wouMs te pe ousde alsverirecef- fas cs.
Sund gfe ire be In th  llir 
----- diversity: 1.2
----- Generating with seed: "light,
I would spread the cloths under your feet:
But I, being poor, have only my dreams;
I have spr"
light,
I would spread the cloths under your feet:
But I, being poor, have only my dreams;
I have spr ths theot tn the
Anwn nd aT wavo ldeEI thase Icour theaI wund i the Tfwherf.
I is lon s no wisr lil d’ce I ri
﻿Cjr fe fe Broud nlecy heon  hre cr cere
Ane amsh Dtshe pe yk cb ld cnoboutte
S yot tirg wJs
yeyave
honne wnoere fargc
mave wave II teayet
Sn pe cever .in﻿ thawni T tec’c wh dnlotehe fns fi hoce pi wh an re cfofJ ls d n. wne w

 wte ceand the ce I syam was theeame have y the cram ceam whe ce was ound te wound ass wtand thave I ls;e the beand whag whe c the ceave cesand thece wand the ce wand theam yous wand th ce was has sand th ce ir I ass whoubsas thave yound thave wha
----- diversity: 1.0
----- Generating with seed: "sy and wanted wear;
Though as for that the passing there
Had worn them really about the same,

And b"
sy and wanted wear;
Though as for that the passing there
Had worn them really about the same,

And boud  hammeramirro thameyh
o wssv thav taee wtecld ths;C tat assy d lalverpe.
Iihun twne fung thavesmy wt uoceam has mwav  tas lhoo wyoo the .ay whed havere  cere founve fo like tas ysold cund tI he t the t the y
hTssra wasa land hag wako; thho thec t asly whe  ans che aslyono the bomendetameame ye pe am wailcbe;Dcere yfne ha theam wtwaverperpsrac beannond eas thro thav thavlsyt anwreae perscre Ts 
----- diversity: 1.2
----- Generating with seed: "sy and wanted wear;
Though as for that the passin

Then took the cram thave wand thave was whav the cram the fand the crave yo wthe crav the ceave yound thave wand tham thave wand thave wand thave wand thave was whav the cram thave wand thave wand thave thave was thaviream whav the cram thave was whav the cram thand thave wand thave wass thave wand thave wand thave was whave wand thave wand the crav the cram thave wand the cram whave wand thave yo thave wand 
----- diversity: 0.5
----- Generating with seed: "ong I stood
And looked down one as far as I could
To where it bent in the undergrowth;

Then took th"
ong I stood
And looked down one as far as I could
To where it bent in the undergrowth;

Then took ths onousw was whund th ce alsy thave tas wherameam the an wtoud theam thave was thea
 wwund thave was wand the cream the crameyyo thave was wham the cravd thave he cram the crav have tha whav  teave whand wass thav the crave hor thaviram yound the croe wand thave wand the fande wand the fand tte fass found thave was found thave wand t

In [28]:

def generate_text(length, diversity): 
    start_index = random.randint(0, len(text) - max_length - 1) 
  
    generated = '' 
    sentence = text[start_index: start_index + max_length] 
    generated += sentence 
    for i in range(length): 
  
            x_pred = np.zeros((1, max_length, len(unique_char))) 
            for t, char in enumerate(sentence): 
                x_pred[0, t, char_to_indices[char]] = 1.
            preds = model.predict(x_pred, verbose = 0)[0] 

            next_index = sample_index(preds, diversity) 
  
            next_char = indices_to_char[next_index] 
  
            generated += next_char 
            sentence = sentence[1:] + next_char 
    return generated 
  
print(generate_text(500, 0.2)) 


the other, as just as fair,
And having perhaps the better claim,
Because it was grassy and wanted weand the cram the cram thave wand thave wand the cram thave was whav the cram thave wand thave wand thave wand the cram the cram the wand thave wand the crave wand the cram thave has thave wand the cround the cround thave wand thave wand thave wand the crave wand thave wand thave wand the crave hand the cram thave wand thave was whav the fand thave wand thave wand thave wand thame wand thavire cand thave wand the cram the cram the cram the cram thave wand thave wand the cram thave yo wthe cround 
